<a href="https://www.kaggle.com/code/rubanzasilva/symptom-disease?scriptVersionId=209438368" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction: Predictive model for differential diagnosis

In this notebook, our goal is to develop a model that can take in a patient's symptoms as an input and return a list of the top 3 possible classes (diseases) alongside confidence values for each class expressed as probabilities.


## Library and Data import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/symptom2disease/Symptom2Disease.csv


In [2]:
%%time
%pip install catboost
%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
%pip install fastkaggle
#%pip install h2o
%pip install -Uqq fastbook
#%pip install polars
#%pip install -q -U autogluon.tabular
#%pip install autogluon
%pip install --upgrade pip
%pip install tqdm
#%pip install wandb
#%pip install sweetviz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [3007 lines of output]
      Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match you

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm

#import fastbook
#fastbook.setup_book()
#from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)
from fastai.text.all import *


from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier,StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold,StratifiedKFold, cross_val_score,train_test_split,GridSearchCV


from pathlib import Path
import os

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv


import warnings


#from openfe import OpenFE, transform
#from autogluon.tabular import TabularDataset, TabularPredictor

#import h2o
#from h2o.automl import H2OAutoML

import gc

import optuna
from optuna.samplers import TPESampler

import pickle
from joblib import dump, load

#import sweetviz as sv
#from IPython.display import FileLink

#import h2o
#from h2o.automl import H2OAutoML

Possible Approaches

1. Using the fastai text transfer learning approach which employs a language model (A model trained to predict the next text in a sentence).
   This employs self-supervised learning, to quote the fastai book
   > Self-supervised learning: Training a model using labels that are embedded in the independent variable, rather than requiring external labels. For instance, training a model to predict the next word in a text.
3. Using traditional tabular models and implementations such as xgboost, to do this we would have to convert our text values (symptoms) to numerical features.

In [4]:
path = Path('/kaggle/input/symptom2disease/')
path

Path('/kaggle/input/symptom2disease')

# Exploratory Data Analysis

In [5]:
!ls /kaggle/input/symptom2disease

Symptom2Disease.csv


In [6]:
df = pd.read_csv(path/'Symptom2Disease.csv',index_col=0)
df.head()

,label,text
0,Psoriasis,"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches."
1,Psoriasis,"My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation."
2,Psoriasis,"I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints."
3,Psoriasis,"There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them."
4,Psoriasis,"My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms."


In [7]:
df.shape

(1200, 2)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 299
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1200 non-null   object
 1   text    1200 non-null   object
dtypes: object(2)
memory usage: 28.1+ KB


In [9]:
#Missing values returned as percentages.
missing_percentages = df.isnull().mean() * 100
sorted_missing_percentages = missing_percentages.sort_values(ascending=False)
print(sorted_missing_percentages)

label    0.0
text     0.0
dtype: float64


In [10]:
df.describe().T

,count,unique,top,freq
label,1200,24,Psoriasis,50
text,1200,1153,"I've been feeling extremely scratchy, sick, and worn out. In addition, I've lost weight and have a temperature. My urine is dark, and my skin has turned yellow. Additionally, I have been experiencing stomach pain.",4


In [11]:
df['label'].nunique()

24

In [12]:
df['label'].unique()

array(['Psoriasis', 'Varicose Veins', 'Typhoid', 'Chicken pox', 'Impetigo', 'Dengue', 'Fungal infection', 'Common Cold',
       'Pneumonia', 'Dimorphic Hemorrhoids', 'Arthritis', 'Acne', 'Bronchial Asthma', 'Hypertension', 'Migraine',
       'Cervical spondylosis', 'Jaundice', 'Malaria', 'urinary tract infection', 'allergy', 'gastroesophageal reflux disease',
       'drug reaction', 'peptic ulcer disease', 'diabetes'], dtype=object)

In [13]:
df['label'].value_counts()

label
Psoriasis                          50
Varicose Veins                     50
peptic ulcer disease               50
drug reaction                      50
gastroesophageal reflux disease    50
allergy                            50
urinary tract infection            50
Malaria                            50
Jaundice                           50
Cervical spondylosis               50
Migraine                           50
Hypertension                       50
Bronchial Asthma                   50
Acne                               50
Arthritis                          50
Dimorphic Hemorrhoids              50
Pneumonia                          50
Common Cold                        50
Fungal infection                   50
Dengue                             50
Impetigo                           50
Chicken pox                        50
Typhoid                            50
diabetes                           50
Name: count, dtype: int64

So there are 24 diseases in this dataset

In [14]:
#X = df['label']
#y = df['text']

In [15]:
#X_train, X_test, y_train, y_test = train_test_split(
    #X, y, test_size=0.20, random_state=42)

In [16]:
#splits = RandomSplitter(valid_pct=0.2)(range_of(df))

# Approach 1 - Text transfer learning
Train a text classifier from a pretrained model


## Using the high-level API

In [17]:
dls = TextDataLoaders.from_df(df, path=path, text_col='text', label_col='label',valid_pct=0.2)

In [18]:
dls.show_batch()

,text,category
0,"xxbos xxmaj i 've had this cough that 's been lingering for days and difficulty breathing . xxmaj my fever is xxunk - high , and xxmaj i 'm feeling so weak and tired . xxmaj i 've also been producing a lot of mucus when i cough , and it 's just been so overwhelming to deal with all of these symptoms . xxmaj i 'm getting so xxunk with all of this .",Bronchial Asthma
1,"xxbos xxmaj i 've had a persistent cough for days , and i also have breathing problems . xxmaj i 'm so weak and exhausted , and my fever is through the roof . xxmaj it 's been very difficult to xxunk all of these symptoms , and xxmaj i 've also been coughing up a lot of mucus . xxmaj i 'm becoming so xxunk over everything .",Bronchial Asthma
2,"xxbos xxmaj mom , xxmaj xxunk , xxmaj i 've been feeling really tired and weak lately , and xxmaj i 've had this cough that just wo n't go away . xxmaj it 's been hard for me to catch my breath , and my fever has been really high . xxmaj when i cough , xxmaj i 've been producing a lot of mucus .",Bronchial Asthma
3,"xxbos xxmaj i 've been feeling really ill lately . xxmaj i 've had this persistent cough and difficulty breathing , and my fever has been off the xxunk . xxmaj i 'm also feeling extremely exhausted , and xxmaj i 've been producing a lot of mucus when i cough . xxmaj it 's just been so overwhelming to deal with all of these symptoms",Bronchial Asthma
4,xxbos xxmaj my eyes are red and watery all the time . xxmaj i 've also had this pressure in my sinuses that wo n't go away . xxmaj i 'm always feeling tired and xxmaj i 've been having a lot of trouble breathing . xxmaj i 've also had a lot of gunk in my throat and my lymph nodes are swollen .,Common Cold
5,"xxbos xxmaj i 've been struggling with difficulty breathing , a constant cough , and fatigue . xxmaj my fever is extremely high , and xxmaj i 've been coughing up a lot of thick , mucoid sputum . xxmaj it 's been so hard to deal with all of this , and xxmaj i 'm just feeling so drained and worn out",Bronchial Asthma
6,"xxbos xxmaj my eyes are always red and itchy , and my nose feels all stuffy and congested . i just feel kind of sick and tired all the time , and i keep coughing up all this gunk . xxmaj my throat feels sore and scratchy , and xxmaj i 've noticed that the bumps on my neck are xxunk than usual",Common Cold
7,xxbos xxmaj i 've been having a really hard time going to the bathroom lately . xxmaj it 's really painful and xxmaj i 'm only able to go every few days . xxmaj there 's also a lot of pain in my anus and around that area . xxmaj my stool has been really bloody and my anus feels really irritated .,Dimorphic Hemorrhoids
8,"xxbos xxmaj i 've been having a tough time with this constant cough , difficulty breathing , and fatigue . xxmaj my fever is high , and xxmaj i 've been coughing up a lot of thick , mucoid sputum . xxmaj it 's all been rather xxunk and exhausting , and xxmaj i 'm feeling quite sick at the xxunk .",Bronchial Asthma


> the library automatically processed all the texts to split then in tokens, adding some special tokens like:

xxbos to indicate the beginning of a te
t
xxmaj to indicate the next word was capitalized

In [19]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [20]:
%%time
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.796918,2.572294,0.458333,00:22


epoch,train_loss,valid_loss,accuracy,time
0,1.862727,1.928242,0.654167,00:52
1,1.500493,0.914594,0.795833,00:51
2,1.157563,0.599242,0.870833,00:52
3,0.918068,0.533003,0.900000,00:52


CPU times: user 6min 49s, sys: 41 s, total: 7min 30s
Wall time: 3min 50s


In [21]:
%%time
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.493947,0.416050,0.904167,00:23


epoch,train_loss,valid_loss,accuracy,time
0,0.363769,0.307160,0.920833,00:52
1,0.294200,0.247419,0.945833,00:52
2,0.228128,0.193011,0.950000,00:51
3,0.181454,0.176671,0.950000,00:52


CPU times: user 6min 50s, sys: 41.3 s, total: 7min 32s
Wall time: 3min 51s


In [22]:
learn.show_results()

,text,category,category_
0,"xxbos xxunk xxunk that xxmaj i 've been experiencing shortness of breath , a persistent cough , and a high fever . xxmaj i 've also been producing a lot of thick , mucoid sputum , and xxmaj i 'm feeling quite drained and exhausted . xxmaj it 's all been rather tough to deal with , and xxmaj i 'm feeling quite sick at the xxunk",Bronchial Asthma,Bronchial Asthma
1,"xxbos xxmaj along with eyesight problems , increased appetite , a sore neck , melancholy , irritability , and regular headaches , i have also been experiencing acid reflux , indigestion , and frequent migraines . xxmaj my regular activities have been made more challenging by these symptoms , which have been causing me a great deal of distress .",Migraine,Migraine
2,"xxbos xxmaj my nose always feels stuffy and congested , and my eyes are always red and itching . i have a feeling of being unwell and fatigued , and i keep hacking up this gunk . i have a scratchy , irritated throat , and xxmaj i 've seen that my neck 's bumps are xxunk than usual .",Common Cold,Common Cold
3,"xxbos i have red , watery eyes all the time . xxmaj my sinuses have also been bothered by a constant tightness . xxmaj my breathing has been quite tough , and xxmaj i 've been feeling worn out all the time . xxmaj my lymph nodes are irritated , and i also have a lot of throat mucous .",Common Cold,Common Cold
4,"xxbos xxmaj recently , xxmaj i 've been scratching myself a lot , and my skin is covered with rashy places . xxmaj additionally , xxmaj i 've had a few pimples that are rather firm , and there are certain spots on my body that have a different shade of xxunk than the rest of my skin .",Fungal infection,Fungal infection
5,"xxbos i have a lot of difficulty breathing . i do n't feel well , and xxmaj i 'm perspiring a lot . xxmaj my chest hurts , and i have a lot of mucus in my throat . xxmaj i 'm having trouble breathing , and the phlegm xxmaj i 'm coughing up has a strange hue .",Pneumonia,Pneumonia
6,"xxbos xxmaj i 've been getting a lot of bowel issues recently . xxmaj it 's difficult for me to go , and it hurts when i do . xxmaj my anus is quite painful , and it has been bleeding whenever i go . xxmaj it 's excruciatingly painful , and xxmaj i 'm quite uneasy .",Dimorphic Hemorrhoids,Dimorphic Hemorrhoids
7,"xxbos xxmaj i 'm feeling really tired and weak lately , and xxmaj i 've had this cough that just wo n't go away . xxmaj my breathing has become shallow , and my fever has been really high . xxmaj when i cough , xxmaj i 've been producing a lot of thick , mucoid sputum",Bronchial Asthma,Bronchial Asthma
8,"xxbos xxmaj my cough and cold have been awful for days . xxmaj my sinuses are congested , and my face is under strain . xxmaj additionally , xxmaj i 've been coughing up phlegm , and it hurts in my chest . i have no sense of smell , and my muscles hurt a lot .",Common Cold,Common Cold


### Inference

In [23]:
learn.predict("I am having a running stomach, fever, general body weakness and have been getting bitten by mosquitoes often")

('Migraine',
 tensor(13),
 tensor([2.5839e-02, 3.2533e-03, 1.8376e-02, 4.3547e-02, 1.1407e-04, 9.5791e-04,
         3.2356e-05, 1.1597e-04, 8.9161e-03, 1.9059e-01, 6.0930e-03, 1.8095e-02,
         5.7278e-03, 6.2621e-01, 1.2189e-02, 3.5489e-04, 3.4870e-04, 1.8086e-04,
         6.9506e-03, 2.0977e-03, 1.2278e-03, 1.6277e-02, 1.7458e-03, 1.0755e-02]))

In [24]:
learn.predict("xxbos i have a lot of itching all over my skin , and sometimes it gets really red and bumpy . xxmaj there are also some patches that are different colors than the rest of my skin , and sometimes i get these little bumps that look like little balls . xxmaj it 's really annoying and itchy.Fungal ")

('Fungal infection',
 tensor(8),
 tensor([1.6129e-02, 3.0662e-04, 1.0455e-04, 1.8343e-04, 5.8483e-03, 9.0106e-05,
         1.3545e-04, 8.3166e-04, 9.6558e-01, 1.2622e-05, 6.7586e-04, 1.7749e-04,
         2.6550e-04, 7.4745e-04, 3.3875e-06, 3.6825e-03, 2.6459e-05, 4.7138e-04,
         1.4743e-03, 8.0851e-04, 1.5150e-03, 6.8964e-05, 3.4225e-04, 5.1738e-04]))

In [25]:
malaria_df = df[df['label'] == 'Malaria']
malaria_df.head()

,label,text
250,Malaria,"I've been experiencing severe itching, chills, vomiting, and a high fever. I'm also sweating a lot and have a headache. I feel nauseous and my muscles are really sore"
251,Malaria,"I've had a high temperature, vomiting, chills, and intense itching. I also have a headache and am perspiring a lot. I'm queasy, and my muscles hurt a lot."
252,Malaria,"I've had a high temperature, vomiting, chills, and intense itching. I also have a headache and am perspiring a lot. I'm queasy, and my muscles hurt a lot."
253,Malaria,"Strong itchiness, chills, nausea, and a high temperature have been plaguing me. Besides having a headache, I'm also perspiring a lot. My muscles are quite sore, and I feel sick."
254,Malaria,"Along with my intense scratching, chills, nausea, and high fever, I've also been feeling really unwell. Along with my headache and excessive sweating, My muscles hurt a lot, and I feel queasy."


### Trying to return probabilities

In [26]:
print(len(learn.dls.vocab))

2


In [27]:
print(learn.dls.vocab)

[['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxrep', 'xxwrep', 'xxup', 'xxmaj', '.', 'and', 'i', ',', 'my', 'a', 'been', 'have', "'ve", 'of', 'to', 'the', 'is', 'in', 'has', 'also', 'it', 'skin', 'lot', 'are', 'really', 'pain', 'on', "'m", 'with', 'fever', 'that', 'me', 'feel', 'feeling', 'experiencing', 'high', 'neck', "'s", 'there', 'having', 'all', 'had', 'for', 'am', 'cough', 'when', 'chest', 'get', 'as', 'quite', "n't", 'headache', 'up', 'throat', 'rash', 'hurts', 'red', 'severe', 'body', 'some', 'itching', 'do', 'or', 'this', 'over', 'back', 'painful', 'weak', 'chills', 'very', 'from', 'difficult', 'like', 'stomach', 'discomfort', 'lost', 'nausea', 'time', 'frequently', 'coughing', 'uncomfortable', 'muscles', 'recently', 'go', 'occasionally', 'legs', 'exhausted', 'ca', 'breathing', 'appetite', 'nose', 'trouble', 'vomiting', 'weight', 'rashes', 'out', 'stiff', 'joints', 'additionally', 'arms', 'swollen', 'yellow', 'pimples', 'itchy', 'these', 'which', 'addition', 'well', 'along

In [28]:
def get_top_3_predictions(text, learn):
    # Get prediction and probabilities
    _, _, probs = learn.predict(text)
    
    # Get the disease labels vocabulary (second list in vocab)
    disease_vocab = learn.dls.vocab[1]  # Access the disease labels
    
    # Get number of classes
    n_classes = len(disease_vocab)
    
    # Get indices of top 3 (or fewer) probabilities
    n_preds = min(3, n_classes)
    top_k_indices = probs.argsort(descending=True)[:n_preds]
    
    # Get the actual labels and their probabilities
    predictions = []
    for idx in top_k_indices:
        label = disease_vocab[int(idx)]  # Use disease_vocab instead of vocab
        probability = float(probs[idx])
        predictions.append((label, f"{probability:.3f}"))
    
    return predictions

def predict_batch_top_3(texts, learn):
    predictions = []
    for text in texts:
        preds = get_top_3_predictions(text, learn)
        predictions.append(preds)
    return predictions

# Test with a single prediction first
test_text = "severe headache with light sensitivity"
single_pred = get_top_3_predictions(test_text, learn)
print("\nSingle prediction test:")
for i, (disease, prob) in enumerate(single_pred, 1):
    print(f"{i}. {disease}: {prob}")

# Then test batch predictions
test_texts = ["severe headache with light sensitivity", 
              "itchy red rash with scaling",
              "frequent urination and excessive thirst"]

results = predict_batch_top_3(test_texts, learn)

print("\nBatch predictions:")
for i, preds in enumerate(results):
    print(f"\nText {i+1}:")
    for j, (disease, prob) in enumerate(preds, 1):
        print(f"{j}. {disease}: {prob}")


Single prediction test:
1. Pneumonia: 0.344
2. Migraine: 0.280
3. urinary tract infection: 0.228



Batch predictions:

Text 1:
1. Pneumonia: 0.344
2. Migraine: 0.280
3. urinary tract infection: 0.228

Text 2:
1. Pneumonia: 0.433
2. Migraine: 0.420
3. Malaria: 0.039

Text 3:
1. Migraine: 0.524
2. urinary tract infection: 0.415
3. diabetes: 0.057


In [29]:
test_texts = ["severe headache with light sensitivity"]
_, _, probs = learn.predict(test_texts)

In [30]:
probs

tensor([3.4923e-03, 1.2268e-05, 2.7063e-03, 2.2166e-05, 5.6909e-07, 1.5337e-04,
        8.4429e-05, 3.0618e-05, 4.7647e-04, 7.7262e-03, 7.3938e-03, 1.0478e-04,
        3.2302e-03, 3.0342e-01, 1.0630e-01, 3.1464e-05, 4.4755e-06, 1.2740e-04,
        5.9700e-04, 6.6601e-02, 7.6663e-05, 2.1190e-03, 1.9100e-05, 4.9526e-01])

In [31]:
# Then test batch predictions
test_texts = ["severe headache with light sensitivity"]

results = predict_batch_top_3(test_texts, learn)

print("\nBatch predictions:")
for i, preds in enumerate(results):
    print(f"\nText {i+1}:")
    for j, (disease, prob) in enumerate(preds, 1):
        print(f"{j}. {disease}: {prob}")


Batch predictions:

Text 1:
1. Pneumonia: 0.344
2. Migraine: 0.280
3. urinary tract infection: 0.228


In [32]:
# Then test batch predictions
test_texts = ["Strong itchiness, chills, nausea, and a high temperature have been plaguing me"]

results = predict_batch_top_3(test_texts, learn)

print("\nBatch predictions:")
for i, preds in enumerate(results):
    print(f"\nText {i+1}:")
    for j, (disease, prob) in enumerate(preds, 1):
        print(f"{j}. {disease}: {prob}")


Batch predictions:

Text 1:
1. Malaria: 0.926
2. Migraine: 0.072
3. Jaundice: 0.001


## Using the data block API

In [33]:
#dis = DataBlock(
    #blocks=(TextBlock.from_df('text', seq_len=72), CategoryBlock),
    #get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

In [34]:
dis_x = DataBlock(
    blocks=(TextBlock.from_df('text'), CategoryBlock),
    get_x=ColReader('text'),
    get_y=ColReader('label'),
    splitter=RandomSplitter(valid_pct=0.2, seed=42)
)

In [35]:
dls = dis_x.dataloaders(df, bs=64)
dls.show_batch(max_n=2)

,text,category
0,"xxbos xxmaj i 've had this cough that 's been lingering for days and difficulty breathing . xxmaj my fever is xxunk - high , and xxmaj i 'm feeling so weak and tired . xxmaj i 've also been producing a lot of mucus when i cough , and it 's just been so overwhelming to deal with all of these symptoms . xxmaj i 'm getting so xxunk with all of this .",Bronchial Asthma
1,"xxbos xxmaj i 've had a persistent cough for days , and i also have breathing problems . xxmaj i 'm so weak and exhausted , and my fever is through the roof . xxmaj it 's been very difficult to xxunk all of these symptoms , and xxmaj i 've also been coughing up a lot of mucus . xxmaj i 'm becoming so xxunk over everything .",Bronchial Asthma
